In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker


In [2]:
# Variables

col_dep = ['Time (s)', 'Potential vs Ag/AgCl (V)', 'Current (A)', 'Charge (C)']
col_cv = ['Potential vs Li$^+$/Li (V)', 'Current (A)', 'Charge (C)', 'Time (s)', 'Current Density ($\mu$A/cm$^2$)', 'Scan']
potentials = ['{:.2f}'.format(round(i, 4))  for i in np.arange(3.90, 4.20, 0.05)]
formatter_0 = ticker.FuncFormatter(lambda x, pos: "{:.1f}".format(x * 1e3))
formatter = ticker.FuncFormatter(lambda x, pos: "{:.1f}".format(x * 1e6))
palette = 'cividis_r'

# Set up files for procedure 'Galv_Cyc_LMO_1C_Final_EIS_20230724.nox'

def setup_files(path):

    files = [os.path.join(path, i) for i in os.listdir(path)]
    folders = ['CV', 'lith', 'delith', 'EIS']
    
    for i in folders:
        if not os.path.exists(os.path.join(path, i)):
            os.mkdir(os.path.join(path, i))

    if not os.path.exists(os.path.join(path, 'EIS', 'Relaxis')):
        os.mkdir(os.path.join(path, 'EIS', 'Relaxis'))
        
    for i in os.listdir(path):
        if 'EIS_' in i:
            os.rename(os.path.join(path, i), os.path.join(path, 'EIS', i))
            
    for i in os.listdir(os.path.join(path, 'EIS')):
        if 'relaxis' in i:
            os.rename(os.path.join(path, 'EIS', i), os.path.join(path, 'EIS', 'Relaxis', i))

    for i in os.listdir(path):
        if '_lith_' in i:
            os.rename(os.path.join(path, i), os.path.join(path, 'lith', i))
        elif '_delith' in i:
            os.rename(os.path.join(path, i), os.path.join(path, 'delith', i))
        elif i != 'README.txt' and len(i) > 7:
            os.rename(os.path.join(path, i), os.path.join(path, 'CV', i))
            
    for file_path in [os.path.join(os.path.join(path, 'EIS', 'Relaxis'), i) for i in os.listdir(os.path.join(path, 'EIS', 'Relaxis'))]:
        # Extract the directory and filename from the file path
        directory, filename = os.path.split(file_path)
        
        # Extract the numeric part from the filename (assuming it's the last part before '_relaxis.txt')
        numeric_part = filename.split('_relaxis.txt')[0].split('EIS_')[-1]
        
        # Convert the numeric part to a float and format it to have 2 decimals
        numeric_with_two_decimals = '{:.2f}'.format(float(numeric_part))
        
        # Create the new filename by replacing the numeric part
        new_filename = filename.replace(numeric_part, numeric_with_two_decimals)
        
        # Create the new file path
        new_file_path = os.path.join(directory, new_filename)
    
        # # Rename the file
        if file_path != new_file_path:
            
            os.rename(file_path, new_file_path)
            print(f"Renamed: {file_path} => {new_file_path}")
       
# Methods
       
def cc(path, area, thickness, sample):

    cv_path = os.path.join(path, 'CV')
    lith_path = os.path.join(path, 'lith')
    delith_path = os.path.join(path, 'delith')
    cv_df = pd.DataFrame()
    
    for i in sorted(os.listdir(cv_path), reverse=True): # sorted to get initial first
        f = os.path.join(cv_path, i)
        df = pd.read_csv(f, sep = ';', skiprows = 1, names = col_cv)
        if '_Initial_' in i:
            df['CV'] = 'Initial'
        else:
            df['CV'] = 'Final'
        cv_df = pd.concat([cv_df, df], ignore_index=True)
        
    lithiation = pd.DataFrame()
    delithiation = pd.DataFrame()
    capacity_l = pd.DataFrame(columns = ['Capacity (mAh/cm$^3$)', 'Cycle', 'L/D'])
    capacity_d = pd.DataFrame(columns = ['Capacity (mAh/cm$^3$)', 'Cycle', 'L/D'])
    efficiency = pd.DataFrame()
    l = [os.path.join(lith_path, i) for i in os.listdir(lith_path)]
    n = 1
    for i in l:
        df = pd.read_csv(i, sep = ';')
        df = df.rename(columns={'WE(1).Potential (V)': col_cv[0], 'Capacity (mAh/cm3)': 'Capacity (mAh/cm$^3$)'})
        df['Cycle'] =  str(n) ### string or int
        df['Capacity_calc'] = -df['WE(1).Charge (C)'] /3.6 / (area * 1e-7 * thickness)
        lithiation = pd.concat([lithiation, df], ignore_index=True)
        capacity_l.loc[n] = (max(df['Capacity_calc']), n, 'Lithiation')
        n += 1
    
    d = [os.path.join(delith_path, i) for i in os.listdir(delith_path)]
    n = 1
    for i in d:
        df = pd.read_csv(i, sep = ';')
        df = df.rename(columns={'WE(1).Potential (V)': col_cv[0], 'Capacity (mAh/cm3)': 'Capacity (mAh/cm$^3$)'})
        df['Cycle'] =  n #str(n) ### string or int
        df['Capacity_calc'] = df['WE(1).Charge (C)'] /3.6 / (area * 1e-7 * thickness)
        delithiation = pd.concat([delithiation, df], ignore_index=True)
        capacity_d.loc[n] = (max(df['Capacity_calc']), n, 'Delithiation')
        n += 1
        
    capacity = pd.concat([capacity_l, capacity_d], ignore_index=True)
    efficiency['Efficiency (%)'] = capacity_l['Capacity (mAh/cm$^3$)'] / capacity_d['Capacity (mAh/cm$^3$)'] * 100
    efficiency['Cycle'] = capacity_l['Cycle']
    # ###
    results = [cv_df, lithiation, delithiation, capacity, efficiency]#, eff]#, efficiency]
    for i in results:
        i['Sample'] = sample
    # ###
    return results#results #cv_df, lithiation, delithiation, capacity, efficiency

def eis(path, sample):
    
    col = ["Frequency (Hz)", "Z'", "-Z''", "Fit Frequency (Hz)", "Fit Z'", "Fit -Z''" ]
    
    files = [os.path.join(path, i) for i in os.listdir(path)]
    eis_data = pd.DataFrame()
    spectra = pd.DataFrame()
    m = 0
    for i in files:
        if 'Fit' in i:
            df = pd.read_csv(i, sep = '\t', skiprows = 2)
            df = df.rename(columns={'FreeVariable' : col_cv[0], 
                                    'Resistance 1: value' : 'Electrolyte Resistance ($\Omega$)', 
                                    'Resistance 2: value' : 'Charge Transfer Resistance ($\Omega$)',
                                    'CPE Q 1: value' : 'Double Layer Capacitance ($\mu$F)'})
            eis_data = pd.concat([eis_data, df], ignore_index=True)
            

        if 'Spectra' in i:
            df_s = pd.read_csv(i, skiprows=4, sep = '\t', names=col)

            df_s[col[2]] = -df_s[col[2]]
            df_s[col[5]] = -df_s[col[5]]
            df_s[col_cv[0]] = potentials[m]
            spectra = pd.concat([spectra, df_s], ignore_index=True)
            m += 1
            
    #     k += 1
    eis_data['Sample'] = sample
    spectra['Sample'] = sample
    eis = [eis_data, spectra]
    return eis #eis_data, spectra

def cc_plot(cc_df, sample, dpi):
    # Plots
    fig, ax = plt.subplots(1, 2, figsize = (15,5), dpi = dpi)
    sns.scatterplot(data = cc_df[0], x = col_cv[0], y = col_cv[4], edgecolor = None, s =1, ax = ax[0], hue = 'CV', palette = palette)
    sns.lineplot(data = cc_df[1], x = 'Capacity_calc', y = col_cv[0], hue = 'Cycle', palette = 'cividis_r')
    sns.lineplot(data = cc_df[2], x = 'Capacity_calc', y = col_cv[0], hue = 'Cycle', legend = False, palette = palette )
    ax[0].yaxis.set_major_formatter(formatter)
    ax[1].set_xlabel('Capacity (mAh/cm$^3$)')
    fig.suptitle(sample)
    

    fig, ax = plt.subplots(1, 2, figsize = (15,5), dpi = dpi)
    sns.scatterplot(data = cc_df[3], x = 'Cycle', y = 'Capacity (mAh/cm$^3$)', hue = 'L/D', palette=palette, ax = ax[0])
    sns.scatterplot(data = cc_df[4], x = 'Cycle', y = 'Efficiency (%)', ax = ax[1])

def eis_plot(eis_df, dpi):
    fig, ax = plt.subplots(dpi = dpi)
    sns.barplot(data = eis_df[0], x = 'Potential vs Li$^+$/Li (V)', y = 'Charge Transfer Resistance ($\Omega$)', palette = palette)

In [3]:
# Potentiostatic N_01

sample = 'Potentiostatic Dep N_01'
path = r'C:\Users\lopezb41\OneDrive - imec\Documents\Experiments\Data\Electrochemical\L21\L21_N_01'
# setup_files(path) DONE
area = np.pi *0.5**2
thick = 90 #nm
n_01 = cc(path, area, thick, sample)
# cc_plot(n_01, sample, 100)

In [4]:
# Galvanostatic N_02 0.03mA/cm2

sample = 'Galvanostatic Dep N_02'
path = r'C:\Users\lopezb41\OneDrive - imec\Documents\Experiments\Data\Electrochemical\L21\L21_N_02'
# setup_files(path)
area = np.pi *0.5**2
thick = 90 #nm
n_02 = cc(path, area, thick, sample)
# cc_plot(n_02, sample, 100)